## Outil de concaténation et de classification par cohortes des courbes de charge
---

In [46]:
import pandas as pd
import os

### Importation des données

Importation des chemins des fichiers csv d'ENEDIS. Il faut renseigner le chemin du dossier avec toutes les courbes de charge.  

In [56]:
CheminD=input("Chemin du dossier contenant les csv des courbes de charge : ")
def import_fichiers(CheminD):
    Chemin_dossier_CDC=r"{}".format(CheminD)
    chemins = []
    nb=0
    for racine, dirs, fichiers in os.walk(f"{Chemin_dossier_CDC}"):
        for file in fichiers:
            nb+=1
    #La condition if permet de ne choisir que les fichiers csv qui commencent par Enedis et dont la taille est supérieure à 19ko
            if file.endswith(".csv") and file.startswith("Enedis") and os.path.getsize(racine+"\\"+file)>=19000 :
                s = os.path.join(racine, file)
                # dfch=pd.read_csv(s, sep=';', skiprows=2, usecols=['Horodate'])
                # dfch=dfch.set_index('Horodate') 
                # dfch.index = pd.to_datetime(dfch.index,utc=True).tz_convert('Europe/Paris')
                # if pd.to_datetime("2023-01-01").tz_localize('Europe/Paris') and pd.to_datetime("2023-01-20").tz_localize('Europe/Paris') in dfch.index:
                chemins.append(s)
    return nb,chemins

nb,chemins=import_fichiers(CheminD)
print(f"Nombre de fichiers dans le dossier = {nb}\nNombre de fichiers csv d'ENEDIS pris = {len(chemins)}")

Nombre de fichiers dans le dossier = 6044
Nombre de fichiers csv d'ENEDIS pris = 5944


### Supression des doublons

In [61]:
def listes_doublons(chemins):
    l,luniques,ldoublons=[],[],[]
    seen=set()

# création d'une liste de tuples formés des chemins csv des PODs et de leur numéro
    for chemin_csv in chemins:
        num_pod=pd.read_csv(chemin_csv,sep=';',nrows=1,usecols=[0]).loc[0,"Identifiant PRM"]
        l.append((chemin_csv,int(num_pod)))

#création de deux listes : luniques est la liste des tuples sans doublons 
#et ldoublons est la liste des tuples qui étaient doublons dans la liste l et qui ne sont pas dans luniques
    for tup in l:
        if tup[1] not in seen:
            seen.add(tup[1])
            luniques.append(tup)
        else:
            ldoublons.append(tup)

    return ldoublons,luniques

In [62]:
%%time
liste_doublons,liste_pods=listes_doublons(chemins)

Wall time: 34min 1s


### Concaténation, rééchantillonnage temporel et sommation pour chaque cohorte

In [63]:
def concaténationPODS(lchemins):
    #Création d'un dataframe vide dfc avec un dateTimeindex allant de nov 2020 à fev 2023 avec un pas de 30 minutes
    dfc=pd.DataFrame(index=pd.date_range(start=pd.Timestamp('2023-01-01 00:00:00', tz='Europe/Paris'),
                                         end=pd.Timestamp('2023-01-21 00:00:00', tz='Europe/Paris'),freq='D'))
    
    # Importation des données de chaque POD de la cohorte et concaténation dans le dataframe dfc
    for path,num in lchemins:
        df = pd.read_csv(path, sep=';', skiprows=2, usecols=['Horodate', 'Valeur']) 
        df = df.set_index('Horodate') 
        df.index = pd.to_datetime(df.index,utc=True).tz_convert('Europe/Paris')
        df = df.resample('D').mean() # rééchantillonnage au pas demi-horaire de l'horizon temporel
        dfc = pd.concat([dfc, df.rename(columns={'Valeur': num})], axis=1) #Ajout au dataframe vide la colonne des valeur du POD
    return dfc 

In [64]:
%%time
dftot= concaténationPODS(liste_pods)

Wall time: 7min 6s


In [67]:
dftot.index=dftot.index.tz_localize(None)

### Concaténation et exportations des données

Création d'un dataframe contenant les numéros et les chemins des PODs doublons supprimés

In [65]:
dfdoublons=pd.DataFrame({"N° PODS en doubles supprimés":[i[1] for i in liste_doublons],"chemins":[i[0] for i in liste_doublons]})

Création d'un excel avec une feuille contenant dftot et une feuille contenant dfdoublons

In [68]:
%%time
with pd.ExcelWriter(Chemin_dossier_CDC+"\\CdC PODs jaunes.xlsx") as writer:
    dftot.to_excel(writer,sheet_name="CdC", startrow=1)
    dfdoublons.to_excel(writer,sheet_name="Doublons")

Wall time: 1min 12s
